# Final Project
### EE 440 University of Washington
### Jordan White

In [2]:
from tkinter import filedialog
import tkinter as tk
import cv2
import PIL.Image, PIL.ImageTk, PIL.ImageEnhance
import numpy as np

# global variables
MARGIN = 10  # px
MAXDIM = 530

class App():
    def __init__(self, window, window_title, image_path="test.bmp"):
        self.window = window
        self.window.title(window_title)

        
        # Load an image using OpenCV
        self.cv_img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        self.NEWcv_img = self.cv_img.copy()  # for recursive processing
        self.NEWcv_img_modify = None
        
        # Get the image dimensions (OpenCV stores image data as NumPy ndarray)
        self.height, self.width, no_channels = self.cv_img.shape
        
        ''' Image Display Related Code'''   
        # Create a FRAME that can fit the images, BLUE
        self.frame1 = tk.Frame(self.window, width=self.width, height=self.height, bg='blue')
        self.frame1.pack(fill=tk.BOTH)
            
        # Create a CANVAS for original image, YELLOW
        self.canvas0 = tk.Canvas(self.frame1, width=MAXDIM, height=MAXDIM+(3*MARGIN), bg='yellow')
        self.canvas0.pack(side=tk.LEFT)
        
        # Create a CANVAS for changing image, ORANGE
        self.canvas1 = tk.Canvas(self.frame1, width=MAXDIM, height=MAXDIM+(3*MARGIN), bg='orange')
        self.canvas1.pack(side=tk.LEFT)
        
        # Use PIL (Pillow) to convert the NumPy ndarray to a PhotoImage
        self.photoOG = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(self.cv_img))
        self.photo = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(self.cv_img))
        
        # Add a PhotoImage to the Canvas (original)
        self.canvas0.create_image(MAXDIM//2, MAXDIM//2, image=self.photoOG)
        
        # Add a PhotoImage to the Canvas (changing effects)
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
        # Write labels for both images, font/size can be changed
        self.canvas0.create_text(MAXDIM//2, MAXDIM+(2*MARGIN),font="Tahoma 20",text="Original Photo")
        self.canvas1.create_text(MAXDIM//2, MAXDIM+(2*MARGIN),font="Tahoma 20",text="Modified Photo")
        
# ##############################################################################################
# ################################   PARAMETER TOOLBAR   #######################################
# ##############################################################################################

        # Create a FRAME that can fit the features on the left
        self.frame2 = tk.Frame(self.window, width=self.width, height=self.height//2, bg='green')
        self.frame2.pack(side=tk.LEFT, fill=tk.BOTH)

        # Create a FRAME that can fit the features on the right
        self.frame3 = tk.Frame(self.window, width=self.width, height=self.height//2, bg='green')
        self.frame3.pack(side=tk.LEFT, fill=tk.BOTH)
        
        
        # Create a SCALE that lets the user blur the image
        self.scl_blur = tk.Scale(self.frame2, from_=1, to=25, orient=tk.HORIZONTAL, showvalue=1,
                command = self.blur_image, length=400, sliderlength=20, label="Blur", font="Tahoma 16")
        self.scl_blur.pack(anchor=tk.N)
        
        # Create a SCALE that lets the user (de)saturate the image
        self.scl_desat = tk.Scale(self.frame2, from_=-50, to=50, orient=tk.HORIZONTAL, showvalue=1,
                command = self.desat_image, length=400, sliderlength=20, label="Saturation", font="Tahoma 16")
        self.scl_desat.pack(anchor=tk.N)
        
        # Create a SCALE that lets the user invert the image
        self.scl_inv = tk.Scale(self.frame2, from_=-50, to=50, orient=tk.HORIZONTAL, showvalue=1,
                command = self.inv_image, length=400, sliderlength=20, label="Contrast", font="Tahoma 16")
        self.scl_inv.pack(anchor=tk.N)
        
        # Create a SCALE that lets the user warm or cool the image
        self.scl_temp2 = tk.Scale(self.frame3, from_=-20, to=20, orient=tk.HORIZONTAL, showvalue=1,
                command = self.temp_image, length=400, sliderlength=20, label="Temperature", font="Tahoma 16")
        self.scl_temp2.pack(anchor=tk.N)
        
        # Create a SCALE that lets the user add noise to the image
        self.scl_noise = tk.Scale(self.frame3, from_=0, to=20, orient=tk.HORIZONTAL, showvalue=1,
                command = self.noise_image, length=400, sliderlength=20, label="Noise", font="Tahoma 16")
        self.scl_noise.pack(anchor=tk.N)
        
        # Create a SCALE that lets the user change exposure of the image
        self.scl_exp = tk.Scale(self.frame3, from_=-50, to=50, orient=tk.HORIZONTAL, showvalue=1,
                command = self.exp_image, length=400, sliderlength=20, label="Exposure", font="Tahoma 16")
        self.scl_exp.pack(anchor=tk.N)
        
        # create a frame for buttons at the bottom
        self.frame0 = tk.Frame(self.window, height=20, width=200)
        self.frame0.pack(anchor=tk.S) 
        
        self.window.mainloop()

##############################################################################################
#################################  CALLBACK FUNCTIONS  #######################################
##############################################################################################
        
    '''#################################  BLUR  ###############################'''
    # Callback for the "Blur" Scale 
    def blur_image(self, k):
        k = self.scl_blur.get() * 2 + 1  # get value from the corresponding scale
        
        sigma = 0
        im = self.cv_img
        # apply Gaussian blur, creating a new image
        blurred = cv2.GaussianBlur(im, (k,k), sigma)
        
        self.NEWcv_img_modify = blurred
        
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.NEWcv_img_modify))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
    '''#################################  DESATURATE  ###############################'''
    # Callback for the "Desaturate" Scale 
    def desat_image(self, k):
        k = self.scl_desat.get()  # get value from the corresponding scale
        
        if k < 0:
            k = -1 * k
            # creating the greyscale image
            grayImage = np.zeros(self.cv_img.shape)
            R = np.array(self.cv_img[:, :, 0])
            G = np.array(self.cv_img[:, :, 1])
            B = np.array(self.cv_img[:, :, 2])

            R = (R *.299)
            G = (G *.587)
            B = (B *.114)

            avg = (R+G+B)
            grayImage = self.cv_img.copy()

            for i in range(3): grayImage[:,:,i] = avg

            newImage = self.NEWcv_img.copy()
            for i in range(3):
                newImage[:,:,i] = k / 100 * grayImage[:,:,i] + (100 - k) / 100 * newImage[:,:,i]         

            self.NEWcv_img_modify = PIL.Image.fromarray((newImage).astype(np.uint8))
            self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(np.uint8(self.NEWcv_img_modify)))
            self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        else:
            temp = self.cv_img.copy()
            temp = cv2.cvtColor(temp, cv2.COLOR_RGB2HSV)
            temp[:, :, 1] = temp[:, :, 1] + k  # increase saturation with python

            newImage = cv2.cvtColor(temp, cv2.COLOR_HSV2RGB)

            if k == 0: newImage = self.cv_img.copy()

            self.NEWcv_img_modify = PIL.Image.fromarray((newImage).astype(np.uint8))
            self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(np.uint8(self.NEWcv_img_modify)))
            self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)

    '''#################################  Contrast  ###############################'''
    # Callback for the "Contrast" Scale 
    def inv_image(self, k):
        k = self.scl_inv.get()  # get value from the corresponding scale
        temp = self.cv_img.copy()
        
        if k < 0:
            k = -1 * k
            inverted = 255 - temp
            newImage = inverted * (k / 100) + ((100 - k) / 100) * temp
        else:
            hsv = cv2.cvtColor(temp, cv2.COLOR_RGB2HSV)
            hsv2 = hsv
            hsv2[:, :, 1] *= 2  # adjust Saturation

            # convert back from HSV colorspace
            newImage = cv2.cvtColor(hsv2, cv2.COLOR_HSV2RGB)
            newImage = k/100*newImage + (100-k)/100*temp
            
        self.NEWcv_img_modify = PIL.Image.fromarray((newImage).astype(np.uint8))
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(np.uint8(self.NEWcv_img_modify)))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
    '''#################################  TEMPERATURE  ###############################'''
    # Callback for the "Temperature" Scale 
    def temp_image(self, k):
        k = self.scl_temp2.get() * -1  # get value from the corresponding scale

        if k < 0:
            tempImage = self.cv_img.copy()
            tempImage[:, :, 0] = 250

            newImage = self.NEWcv_img.copy()
            newImage[:,:,0] = -1*k / 100 * tempImage[:,:,0] + (100 + k) / 100 * newImage[:,:,0]

            self.NEWcv_img_modify = PIL.Image.fromarray((newImage).astype(np.uint8))
            self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(np.uint8(self.NEWcv_img_modify)))
            self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        else:
            tempImage = self.cv_img.copy()
            tempImage[:, :, 2] = 250

            newImage = self.NEWcv_img.copy()
            newImage[:,:,2] = k / 100 * tempImage[:,:,2] + (100 - k) / 100 * newImage[:,:,2]

            self.NEWcv_img_modify = PIL.Image.fromarray((newImage).astype(np.uint8))
            self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(np.uint8(self.NEWcv_img_modify)))
            self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
    '''#################################  Noise  ###############################'''
    # Callback for the "Noise" Scale 
    def noise_image(self, k):
        k = self.scl_noise.get()  # get value from the corresponding scale

        tempImage = self.cv_img.copy()
        
        row,col,ch= tempImage.shape
        mean = 0.1
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        newImage = tempImage * gauss
        
        newImage = k / 100 * newImage + (100 - k) / 100 * tempImage
        
        self.NEWcv_img_modify = PIL.Image.fromarray((newImage).astype(np.uint8))
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(np.uint8(self.NEWcv_img_modify)))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
    '''#################################  Exposure  ###############################'''
    # Callback for the "Exposure" Scale 
    def exp_image(self, k):
        k = self.scl_exp.get()  # get value from the corresponding scale

        tempImage = self.cv_img.copy()

        # Creating object of Brightness class
        im = PIL.ImageEnhance.Brightness(PIL.Image.fromarray(tempImage))

        # showing resultant image
        newImage = im.enhance((100+k)/100)

        self.NEWcv_img_modify = newImage
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(np.uint8(self.NEWcv_img_modify)))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER) 

##############################################################################################
# Create a window and pass it to the Application object
App(tk.Tk(), "Final Project")